In [2]:
import pandas as pd
import s3fs
from smart_open import open
import boto3
from io import StringIO # python3; python2: BytesIO 
from boto3.s3.transfer import TransferConfig
import metrics
import torch
from transformers import *
import numpy as np

### Reply

In [ ]:
column_of_interest = ["text_ tokens", "engaging_user_id", "reply_timestamp"]
train_set = pd.read_csv('s3://recsys-challenge-2020/training.tsv', encoding="utf-8",  sep="\x01",
                     usecols= [0, 14, 20], names = column_of_interest)
column_of_interest = ["engaging_user_id"]
val_set = pd.read_csv('s3://recsys-challenge-2020/val.tsv', encoding="utf-8", sep="\x01",
                     usecols= [14], names=column_of_interest)
unique_val_set_users = val_set[['engaging_user_id']].drop_duplicates()
train_set_positive = train_set[train_set.reply_timestamp.notnull()]
warm_train_set_positive = pd.merge(train_set_positive, unique_val_set_users, how = 'inner', on = 'engaging_user_id')
user_tokens = warm_train_set_positive.groupby(['engaging_user_id'])['text_ tokens'].agg(list).reset_index()
user_tokens.to_csv('s3://recsys-challenge-2020/user_tokens_reply.csv', index = False)

### Retweet

In [ ]:
column_of_interest = ["text_ tokens", "engaging_user_id", "retweet_timestamp"]
train_set = pd.read_csv('s3://recsys-challenge-2020/training.tsv', encoding="utf-8",  sep="\x01",
                     usecols= [0, 14, 21], names = column_of_interest)
column_of_interest = ["engaging_user_id"]
val_set = pd.read_csv('s3://recsys-challenge-2020/val.tsv', encoding="utf-8", sep="\x01",
                     usecols= [14], names=column_of_interest)
unique_val_set_users = val_set[['engaging_user_id']].drop_duplicates()
train_set_positive = train_set[train_set.retweet_timestamp.notnull()]
warm_train_set_positive = pd.merge(train_set_positive, unique_val_set_users, how = 'inner', on = 'engaging_user_id')
user_tokens = warm_train_set_positive.groupby(['engaging_user_id'])['text_ tokens'].agg(list).reset_index()
user_tokens.to_csv('s3://recsys-challenge-2020/user_tokens_retweet.csv', index = False)

In [2]:
column_of_interest = ["text_ tokens", "engaging_user_id", "reply_timestamp"]
train_set = pd.read_csv('s3://recsys-challenge-2020/training.tsv', encoding="utf-8",  sep="\x01",
                     usecols= [0, 14, 20], names = column_of_interest)

In [3]:
column_of_interest = ["engaging_user_id"]
val_set = pd.read_csv('s3://recsys-challenge-2020/val.tsv', encoding="utf-8", sep="\x01",
                     usecols= [14], names=column_of_interest)

In [11]:
val_set

,engaging_user_id
0,0000006C3074607050F1339DDCB890BB
1,00000776B07587ECA9717BFC301F2D6E
2,00000860E80C67D8C46CE57C64DE9444
3,00000865A1538142CDA5936B07FE4311
4,00000865A1538142CDA5936B07FE4311
...,...
15127679,FFFFFB739796BE92A4278AF12F89A341
15127680,FFFFFE97AA06327403491D71E2ED52DA
15127681,FFFFFE97AA06327403491D71E2ED52DA
15127682,FFFFFE97AA06327403491D71E2ED52DA


In [5]:
unique_val_set_users = val_set[['engaging_user_id']].drop_duplicates()

In [6]:
train_set_positive = train_set[train_set.reply_timestamp.notnull()]

In [7]:
warm_train_set_positive = pd.merge(train_set_positive, unique_val_set_users, how = 'inner', on = 'engaging_user_id')

In [8]:
len(warm_train_set_positive)

1408991

In [9]:
user_tokens = warm_train_set_positive.groupby(['engaging_user_id'])['text_ tokens'].agg(list).reset_index()

In [10]:
user_tokens.to_csv('s3://recsys-challenge-2020/user_tokens_reply.csv', index = False)

In [12]:
len(user_tokens)

865528